In [1]:
#import all librairies we'll need
import json
import requests as r
import pandas as pd
import time

In [2]:
#define all our variables
url='https://search.ubisoft.com/api/v2/search'
headers = """Cache-Control: no-cache
Connection: keep-alive
Content-Length: 205
Content-Type: application/json
Host: search.ubisoft.com
Origin: https://www.ubisoft.com
Pragma: no-cache
Referer: https://www.ubisoft.com/fr-fr/games
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36"""
headers=dict(i.split(': ') for i in headers.split('\n'))
payload="""{"appId":"f35adcb5-1911-440c-b1c9-48fdc1701c68","fields":["title"],"from":0,"languages":["fr-fr"],"size":15,"sorts":[{"field":"createdAt","direction":"desc"}],"channels":["ubisoftportal"],"types":["game"]}"""


In [3]:
#collect data
resp=r.post(url, headers=headers, data=payload)
result = resp.json()
df0 = pd.DataFrame(result) 
df0
#what interests us is in line [4], column [3]

,took,timed_out,_shards,hits
total,5,False,5.0,160
successful,5,False,5.0,NaN
failed,5,False,0.0,NaN
max_score,5,False,NaN,None
hits,5,False,NaN,"[{'_index': 'ubisoftportal_fr-fr_v2', '_type':..."


In [4]:
def get_df0():
    """return 15 lines of data corresponding to 15 first games"""
    
    resp=r.post(url, headers=headers, data=payload)   #collects datas         
    result = resp.json()
    df0 = pd.DataFrame(result)                    #get array with empty boxes except one which contains our dataframe  
    df = pd.DataFrame(df0.iloc[4,3])             #takes the data out of the aforementioned box  
    
    return df

In [5]:
df = get_df0()
df

,_index,_type,_id,_score,_source,sort
0,ubisoftportal_fr-fr_v2,game,293048,None,"{'id': '293048', 'gameInfo': ['<h3 xmlns=""http...",[1656633600000]
1,ubisoftportal_fr-fr_v2,game,363230,None,"{'id': '363230', 'gameInfo': ['<h2 xmlns=""http...",[1614211200000]
2,ubisoftportal_fr-fr_v2,game,362795,None,"{'gameInfo': ['<p xmlns=""http://www.w3.org/199...",[1613606400000]
3,ubisoftportal_fr-fr_v2,game,363254,None,"{'id': '363254', 'gameInfo': ['<h2 xmlns=""http...",[1611187200000]
4,ubisoftportal_fr-fr_v2,game,363231,None,"{'id': '363231', 'gameInfo': ['<h2 xmlns=""http...",[1607990400000]
5,ubisoftportal_fr-fr_v2,game,350318,None,"{'id': '350318', 'gameInfo': [' <h2 xmlns...",[1606953600000]
6,ubisoftportal_fr-fr_v2,game,361679,None,"{'id': '361679', 'gameInfo': ['<h2 xmlns=""http...",[1605571200000]
7,ubisoftportal_fr-fr_v2,game,363096,None,"{'id': '363096', 'gameInfo': ['<p xmlns=""http:...",[1605139200000]
8,ubisoftportal_fr-fr_v2,game,350634,None,"{'id': '350634', 'gameInfo': [' <h2 style...",[1603929600000]
9,ubisoftportal_fr-fr_v2,game,363269,None,"{'id': '363269', 'gameInfo': ['AGOS'], 'mdmBra...",[1603843200000]


In [6]:
def get_df(df,payload=payload):
    """add 15 more lines of data corresponding to 15 other games"""    
    dct = json.loads(payload)
        
    while dct['from'] <300:
        dct['from'] += 15
        payload = str(dct)         #increases the variable used to call the next page   
        
        resp=r.post(url, headers=headers, data=payload)
        result = resp.json()
        df1 = pd.DataFrame(result)                        #get data from next page
        df1 = pd.DataFrame(df1.iloc[4,3])
        
        df = pd.concat([df, df1])         #add the new collected rows to the array
        time.sleep(0.5)                     #wait 1 sec before next request
        
    return df

In [7]:
df = get_df0().pipe(get_df)
df

,_index,_type,_id,_score,_source,sort
0,ubisoftportal_fr-fr_v2,game,293048,None,"{'id': '293048', 'gameInfo': ['<h3 xmlns=""http...",[1656633600000]
1,ubisoftportal_fr-fr_v2,game,363230,None,"{'id': '363230', 'gameInfo': ['<h2 xmlns=""http...",[1614211200000]
2,ubisoftportal_fr-fr_v2,game,362795,None,"{'gameInfo': ['<p xmlns=""http://www.w3.org/199...",[1613606400000]
3,ubisoftportal_fr-fr_v2,game,363254,None,"{'id': '363254', 'gameInfo': ['<h2 xmlns=""http...",[1611187200000]
4,ubisoftportal_fr-fr_v2,game,363231,None,"{'id': '363231', 'gameInfo': ['<h2 xmlns=""http...",[1607990400000]
...,...,...,...,...,...,...
5,ubisoftportal_fr-fr_v2,game,301085,None,"{'id': '301085', 'gameInfo': ['<p xmlns=""http:...",[1166572800000]
6,ubisoftportal_fr-fr_v2,game,357908,None,"{'id': '357908', 'gameInfo': ['<p xmlns=""http:...",[-2208988800000]
7,ubisoftportal_fr-fr_v2,game,357482,None,"{'id': '357482', 'gameInfo': ['<p xmlns=""http:...",[-2208988800000]
8,ubisoftportal_fr-fr_v2,game,200954,None,"{'gameInfo': [' <p xmlns=""http://www.w...",[-2208988800000]


In [8]:
def clean(df):
    """Data manipulation for a better reading of our datas"""
    
    df= df.reset_index(drop=True)     #Order the index

    flattened_source = pd.DataFrame(dict(df['_source'])).T  #get data stored in dictionary form in a single column
    df = pd.concat([df, flattened_source], axis=1)
    df.drop('_source', axis=1, inplace=True)                #drop the aforementioned column
    
    df.drop(['platformInfoList','gameInfo'], axis=1, inplace=True)        #Drop columns containing soup of info which doesn't interest us
    df.drop('_score', axis=1, inplace=True)                                   #Drop empty column
    df.drop(['_index','_type', 'channel', 'language'], axis=1, inplace=True)      #Drop columns which contains same value in all their lines
    df.drop(['_id','name', 'friendlyUrl', 'mdmInstallment'], axis=1, inplace=True)    #Drop twins columns (or containing almost same info between them)

    df = df.rename(columns={'mdmBrand':'brandTeam','boxshotLink':'link_picture1', 'mDMRating':'minumumAge', 'metaKeyword':'keywords','thumbnail':'link_picture2','link':'link-website', 'createdAt':'releaseDate', 'metaDescription':'description'})    
    df = df[['title','id','releaseDate','sort','minumumAge','genre','link_picture1','link_picture2', 'link-website','developers','brandTeam', 'keywords','description']]
    
    return df

In [9]:
df = get_df0().pipe(get_df).pipe(clean)
df

,title,id,releaseDate,sort,minumumAge,genre,link_picture1,link_picture2,link-website,developers,brandTeam,keywords,description
0,Beyond Good and Evil 2,293048,2022-07-01T00:00:00,[1656633600000],[PEGI - 18 - Provisional],"[Action, Adventure]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/beyond-good...,[Ubisoft Montpellier],BEYOND GOOD AND EVIL - 33,"Beyond, Good, Evil, Ubisoft, jeu vidéo, scienc...",NaN
1,Riders Republic,363230,2021-02-25T00:00:00,[1614211200000],[],"[Action, Sports]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/riders-repu...,[],RIDERS REPUBLIC,NaN,NaN
2,Far Cry 6,362795,2021-02-18T00:00:00,[1613606400000],[],"[Action, Adventure, FPS]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/far-cry-6,[Ubisoft Montreal],FAR CRY - 108,"Ubisoft, Far Cry, Far Cry 6","Official Site. In Far Cry 6, play as a local Y..."
3,Prince of Persia: The Sands of Time Remake,363254,2021-01-21T00:00:00,[1611187200000],[],"[Action, Adventure, Platformer]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/prince-of-p...,[Ubisoft Pune],PRINCE OF PERSIA - 205,NaN,NaN
4,Scott Pilgrim vs The World: The Game,363231,2020-12-15T00:00:00,[1607990400000],[],"[Adventure, Arcade]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/scott-pilgr...,[Ubisoft Montreal],Scott Pilgrim - Brand,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Equideow,301085,2006-12-20T00:00:00,[1166572800000],NaN,"[Family, Simulation, Gestion]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/howrse,[Owlient],Howrse - Brand,"Equideow, Howrse, jeux de chevaux, animaux, ra...",Rejoignez la communauté de passionnés de cheva...
156,Lapins Crétins : Short Stories,357908,1900-01-01T00:00:00,[-2208988800000],NaN,[Movies],https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/rabbids-sho...,[],RABBIDS - 1870,Lapin Cretins Short Stories - Ubisoft Film & T...,Ubisoft Film & Television Paris a laissé 10 st...
157,VR Rabbids Ride,357482,1900-01-01T00:00:00,[-2208988800000],[],"[Action, Family, Arcade]",https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/VR-Rabbids-...,"[LAI Games, Ubisoft]",VR PROJECTS - 2353,"Virtuel, Lapins Crétins, Ride, Montagnes russe...",Montez dans les montagnes russes et foncez dan...
158,Rayman Raving Rabbids,200954,1900-01-01T00:00:00,[-2208988800000],NaN,[],https://ubistatic19-a.akamaihd.net/ubicomstati...,https://ubistatic19-a.akamaihd.net/ubicomstati...,https://www.ubisoft.com/fr-fr/game/,"[Ubisoft Montpellier, Ubisoft Bulgaria]",RAVING RABBIDS - 349,NaN,NaN
